### Step 1: Install CANYA

Google Colab 환경에서 Conda 없이 바로 CANYA를 설치합니다. 필요한 종속성(`tensorflow`, `numpy`, `pandas`)은 Colab에 이미 설치되어 있으므로 별도 설치가 필요하지 않습니다.

In [18]:
# --- 1단계: Conda 설치 ---
# Colab에 Conda 환경을 관리할 Miniforge를 설치합니다.
!wget -q "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh"
!bash Miniforge3-Linux-x86_64.sh -bfp /usr/local
!conda config --set auto_update_conda false

# --- 2단계: CANYA 전용 가상 환경 생성 및 모든 의존성 설치 ---
# python 3.8을 기반으로 numpy, pandas, tensorflow 등 모든 요구사항을 한 번에 설치합니다.
!conda create -n canya_env python=3.8 numpy=1.19.5 pandas=1.3.3 tensorflow=2.6.0 keras=2.6.0 "protobuf<=3.20" matplotlib seaborn -c conda-forge -y

# --- 3단계: 가상 환경 내에 CANYA 설치 ---
# 위에서 만든 canya_env 환경 안에서 pip를 사용해 canya를 설치합니다.
!conda run -n canya_env pip install --no-cache-dir https://github.com/lehner-lab/canya/tarball/master

print("\n\n✅ 모든 설치가 성공적으로 완료되었습니다!")
print("CANYA는 'canya_env'라는 가상 환경에 설치되었습니다.")
print("아래 '가상 환경에서 CANYA 실행하기' 설명을 참고하여 다음 단계를 진행하세요.")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  abseil-cpp         conda-forge/linux-64::abseil-cpp-20210324.2-h9c3ff4c_0 
  absl-py            conda-forge/noarch::absl-py-0.15.0-pyhd8ed1ab_0 
  aiohappyeyeballs   conda-forge/noarch::aiohappyeyeballs-2.4.3-pyhd8ed1ab_0 
  aiohttp            conda-forge/linux-64::aiohttp-3.10.5-py38h2019614_0 
  aiosignal          conda-forge/noarch::aiosignal-1.3.1-pyhd8ed1ab_0 
  alsa-lib           conda-forge/linux-64::alsa-lib-1.2.3.2-h166bdaf_0 
  astunparse         conda-forge/noarch::astunparse-1.6.3-pyhd8ed1ab_2 
  async-timeout      conda-forge/noarch::async-timeout-4.0.3-pyhd8ed1ab_0 
  attr               conda-forge/linux-64::attr-2.5.1-h166bdaf_1 
  attrs              conda-forge/noarch::attrs-24.2.0-pyh71513ae_0 
  blinker            conda-forge/noarch::blinker-1.8.2-pyhd8ed1ab_0 
  brotli             conda-forge/linux-64::brotli-1.1.0-hb9d3cd8_3 
  brotli-bin         conda-forge/linux-64::brotli-bin-1.1.0-hb9d3cd8_3 
  brotli-python      conda-forge

### Step 2: Prepare Input File

`canya_input.txt`라는 입력 파일을 생성합니다. 형식은 CANYA에서 요구하는 형식에 맞게 구성해야 합니다 (예: 탭 구분 서열 리스트).

In [23]:
# 헤더("ID\tSequence\n")를 제거하고 파일을 다시 생성합니다.
with open("canya_input.txt", "w") as f:
    f.write("Seq1\tCASSIRSSYEQYF\n")
    f.write("Seq2\tCASSLGQGAETLYF\n")

print("헤더가 제거된 'canya_input.txt' 파일을 생성했습니다.")

헤더가 제거된 'canya_input.txt' 파일을 생성했습니다.


### Step 3: Run CANYA (Basic Mode)

In [22]:
# !conda run -n canya_env 접두사를 붙여서 실행합니다.
print("--- Running Basic Mode ---")
!conda run -n canya_env canya --input canya_input.txt --output basic_out.txt

print("\n--- Running Summarize Mode ---")
!conda run -n canya_env canya --input canya_input.txt --output summarized_out.txt --summarize mean

print("\n--- Running Ensemble Mode ---")
!conda run -n canya_env canya --input canya_input.txt --output ensemble_out.txt --summarize mean --mode ensemble

print("\n\n✅ 모든 실행이 완료되었습니다. 출력 파일을 확인하세요.")

Loaded model models/model_weights1.h5
Read input

2025-07-12 11:05:14.873546: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-07-12 11:05:14.873582: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-07-12 11:05:14.873609: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c4df3f62770a): /proc/driver/nvidia/version does not exist
2025-07-12 11:05:14.873993: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the approp

### Step 6: Visualize Results

In [14]:
# 전체 파이썬 코드를 !conda run -n canya_env python -c "..." 형태로 실행합니다.
!conda run -n canya_env python -c " \
import pandas as pd; \
import matplotlib.pyplot as plt; \
import seaborn as sns; \
import sys; \
\
try: \
    df = pd.read_csv('ensemble_out.txt', sep='\\t'); \
    df_sorted = df.sort_values('Mean_Score', ascending=False).reset_index(drop=True); \
    \
    plt.figure(figsize=(10, 6)); \
    sns.barplot(x='ID', y='Mean_Score', data=df_sorted, palette='viridis', edgecolor='black'); \
    plt.errorbar(x=df_sorted.index, y=df_sorted['Mean_Score'], yerr=df_sorted['Std_Dev_Score'], fmt='none', c='black', capsize=5); \
    plt.title('CANYA Nucleation Score (Ensemble Mode)', fontsize=16); \
    plt.xlabel('Sequence ID'); \
    plt.ylabel('Mean ± Std Dev'); \
    plt.ylim(0, 1); \
    plt.tight_layout(); \
    plt.savefig('canya_results.png'); \
    print('Result plot saved to canya_results.png'); \
except FileNotFoundError: \
    print('Error: ensemble_out.txt not found. Please ensure the previous CANYA command ran successfully.', file=sys.stderr); \
except Exception as e: \
    print(f'An error occurred: {e}', file=sys.stderr) \
"

Error reading results: [Errno 2] No such file or directory: 'ensemble_out.txt'
